<a id="home"></a>
## Table of content
1. [EDA](#eda)
    1. [Labels]
2. [Modeling](#modeling)

<a id="eda"></a>
## EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<a id="labels"></a>
[home](#home)

<a id="modeling"></a>
## Modeling

<a id="labels"></a>
[home](#home)